In [1]:
import re
import pandas as pd
from helpers import sql

# pandas formatting
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)
pd.set_option('display.float_format', '{:.0f}'.format)

In [2]:
meetings = sql("SELECT * FROM csas2_meeting")
documents = sql("""
    SELECT * FROM csas2_document
        JOIN csas2_documenttype ON csas2_document.document_type_id = csas2_documenttype.id
        JOIN csas2_document_meetings ON csas2_document.id = csas2_document_meetings.document_id;
""", database='dmapps_250723')
processes = sql("""
    SELECT * FROM csas2_process
        JOIN csas2_csasoffice ON csas2_process.lead_office_id = csas2_csasoffice.id
        JOIN shared_models_region ON csas2_csasoffice.region_id = shared_models_region.id;
""", database='dmapps_250723')

In [3]:
# image of names with no id or pub numbers, typed manually
fsar_pilots = [
    ('Pacific Ocean Perch', 'PAC'),
    ('Salmon (7 SMUs)', 'PAC'),
    ('Dolly Varden', 'Arctic'),
    ('EAZ/WAZ Shrimp', 'Arctic'),
    ('4RST Capelin', 'QC'),
    ('Unit 1 & 2 Redfish', 'QC'),
    ('Lobster LFA 35', 'MAR'),
    ('4TVn Atlantic Cod', 'GULF'),
    ('4T Spring and Fall Herring', 'GULF'),
    ('Gulf of St. Lawrence Snow Crab', 'GULF'),
    ('Southern Gulf of St. Lawrence Scallop', 'GULF'),
    ('3Ps Atlantic Cod', 'NL'),
    ('Northern Shrimp SFA 4-6', 'NL'),
    ('Northern Cod', 'NL'),
]
fsar_srr_2024 = [
    '2024/016',
    '2024/017',
    '2024/039',
    '2024/040',
    '2024/041',
]
fsar_sar_2024 = [
    '2024/012',
    '2024/016',
    '2024/020',
    '2024/009',
    '2024/021',
    '2024/026',
    '2024/030',
    '2024/029',
    '2024/037',
    '2024/049',
    '2024/055',
    '2024/057',
    '2024/061',
]
fsar_srr_2025 = [
    '2025/001',
    '2025/002',
    '2025/003',
    '2025/008',
    '2025/009',
    '2025/010',
    '2025/011',
    '2025/012',
    '2025/015',
    '2025/016',
    '2025/019',
    '2025/020',
    '2025/021',
]
fsar_sar_2025 = [
    '2025/007',
    '2025/009',
    '2025/010',
    '2025/011',
    '2025/012',
    '2025/013',
    '2025/014',
    '2025/017',
    '2025/018',
    '2025/022',
    '2025/024',
    '2025/026',
    '2025/029',
]
# raw data from the spreadsheet, cleaned the best I can
fsar_coordinators = [
    # NL
    'SAR 2024-016',
    'SAR 2024/061',
    'SAR 2024_049',
    'CSAS - Process 761 - 3Ps cod ',
    'CSAS - Process 756 - 3KLPs herring',
    'CSAS - Process 757 - Snow Crab ',
    'CSAS - Process 758 - Capelin',
    'CSAS - Process 759 - Striped and Northern Shrimp ',
    'CSAS - Process 795 - Northern cod ',
    # QC
    'SAR 2024/021',
    'SR2024/008',
    'SR2025/002',
    'SAR2025/017',
    'SCAS - Processus 807 - Estuary and Northern Gulf Snow crab',
    'SAR2025/011',
    'SAR2025/012',
    'SAR2025/013',
    'SAR2025/009',
    'SAR2025/014',
    'SAR2025/024',
    'SR2025/016',
    'SR2025/015',
    # ON (n/a)
    # ARCTIC
    'Great Slave Lake Inconnu (Stenodus leucichthys) Stock Assessment, 2022',
    'Great Slave Lake Lake Whitefish Stock Assessment, 2022',
    'Great Slave Lake Lake Trout Stock Assessment, 2022',
    # PACIFIC (n/a)
    # GULF
    'https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_026-eng.html',
    'Science Advisory Report 2024/029',
    'Science Advisory Report 2024/030',
    'Science Advisory Report 2024/037',
    'SAR 2024/055',
    '4T striped bass',
    '4T groundfish',
    '4T snow crab',
    '4T salmon',
    # MAR
    'SAR 2024/056',
    'SAR 2024/057',
    'SR 2024/039',
    'SR 2024/040',
    'SR 2025/008',
    'Western Component Pollock Management Strategy Evaluation Interim Update',
    'SR 2025/009',
    '2024 Update of the Eastern Scotian Shelf Shrimp Stock Assessment',
    'Assessment of Maritimes Region Snow Crab (N-ENS (CFA 20-22), S-ENS(CFA 23 and CFA 24) and 4X)',
    'LFA 35 Lobster Stock Status Update',
    'LFA 36 Lobster Stock Status Update', 
    'LFA 38 Lobster Stock Status Update',
]  
fsar_coordinators_sar = [
    '2024/016',
    '2024/061',
    '2024/049',
    '2024/021',
    '2025/017',
    '2025/011',
    '2025/012',
    '2025/013',
    '2025/009',
    '2025/014',
    '2025/024',
    '2024/026',
    '2024/029',
    '2024/030',
    '2024/037',
    '2024/055',
    '2024/056',
    '2024/057',
]
fsar_coordinators_srr = [
    '2024/008',
    '2025/002',
    '2025/016',
    '2025/015',
    '2024/039',
    '2024/040',
    '2025/008',
    '2025/009',
]
fsar_coordinators_process_id = [
    761,
    756,
    757,
    758,
    759,
    795,
    807,
    
    # added based on MAR requests below (fsar_requests)
    782,
    750,
    784,
    777,
    
]
# doc name doesn't generally match the name used by the process or document
fsar_coordinators_doc_titles_approx = [
    # ARCTIC
    ('Great Slave Lake Inconnu (Stenodus leucichthys) Stock Assessment, 2022', 'Arctic'),
    ('Great Slave Lake Lake Whitefish Stock Assessment, 2022', 'Arctic'),
    ('Great Slave Lake Lake Trout Stock Assessment, 2022', 'Arctic'),
    # GULF
    ('4T striped bass', 'GULF'),
    ('4T groundfish', 'GULF'),
    ('4T snow crab', 'GULF'),
    ('4T salmon', 'GULF'),
    # MAR - these are all requests based on links in spreadsheet
    # ('Western Component Pollock Management Strategy Evaluation Interim Update', 'MAR'),
    # ('2024 Update of the Eastern Scotian Shelf Shrimp Stock Assessment', 'MAR'),
    # ('Assessment of Maritimes Region Snow Crab (N-ENS (CFA 20-22), S-ENS(CFA 23 and CFA 24) and 4X)', 'MAR'),
    # ('LFA 35 Lobster Stock Status Update', 'MAR'),
    # ('LFA 36 Lobster Stock Status Update', 'MAR'),
    # ('LFA 38 Lobster Stock Status Update', 'MAR'),
]
fsar_requests = [407, 464, 433, 449, 446, 446] # from MAR request links
# note: requests 449/446 are both process 777 
# process 777 has 3 meetings: 1441, 1442, 1443 all stock status updates for lobster (35, 36, 38)


# docs from pub numbers

In [4]:
doc_columns = ['id', 'title_en', 'pub_number', 'process_id', 'acronym_en', ]

fsar_pub_number_lists = [
    [fsar_srr_2024, 'SRR'],
    [fsar_sar_2024, 'SAR'],
    [fsar_srr_2025, 'SRR'],
    [fsar_sar_2025, 'SAR'],
    [fsar_coordinators_srr, 'SRR'],
    [fsar_coordinators_sar, 'SAR'],
]

In [5]:
# confirm that all fsar docs are accounted for and single counted - GOOD
for fsar_list, doc_type in fsar_pub_number_lists:
    temp_docs = documents[
        (documents.pub_number.isin(fsar_list)) & (documents.acronym_en == doc_type)
    ]
    print(temp_docs.shape[0] == len(fsar_list))

True
True
True
True
True
True


In [6]:
# confirm that none of the docs are on multiple lists - BAD
len([t + y for x, t in fsar_pub_number_lists for y in x]), len({t + y for x, t in fsar_pub_number_lists for y in x})

# all duplication comes from the coordinators list, but that list also includes 2 docs that aren't in the original list

(70, 46)

In [7]:
all_fsar_pub_numbers = [(t, y) for x, t in fsar_pub_number_lists for y in x]

In [8]:
fsar_doc_id = list()

for abbrev, pub_number in all_fsar_pub_numbers:
    id_to_append = int(documents[
        (documents.pub_number == pub_number) & (documents.acronym_en == abbrev)
    ].id.values[0])
    print(abbrev, pub_number, id_to_append)
    fsar_doc_id.append(id_to_append)
fsar_doc_id = sorted(list(set(fsar_doc_id)))
len(fsar_doc_id)

SRR 2024/016 1230
SRR 2024/017 1229
SRR 2024/039 1346
SRR 2024/040 1347
SRR 2024/041 1348
SAR 2024/012 1123
SAR 2024/016 1211
SAR 2024/020 1187
SAR 2024/009 1172
SAR 2024/021 1274
SAR 2024/026 1213
SAR 2024/030 1250
SAR 2024/029 1251
SAR 2024/037 1272
SAR 2024/049 1255
SAR 2024/055 1326
SAR 2024/057 1316
SAR 2024/061 1270
SRR 2025/001 1359
SRR 2025/002 1373
SRR 2025/003 1302
SRR 2025/008 1378
SRR 2025/009 1379
SRR 2025/010 1377
SRR 2025/011 1376
SRR 2025/012 1380
SRR 2025/015 1446
SRR 2025/016 1444
SRR 2025/019 1417
SRR 2025/020 1382
SRR 2025/021 1303
SAR 2025/007 1352
SAR 2025/009 1419
SAR 2025/010 1398
SAR 2025/011 1416
SAR 2025/012 1425
SAR 2025/013 1424
SAR 2025/014 1445
SAR 2025/017 1415
SAR 2025/018 1355
SAR 2025/022 1426
SAR 2025/024 1466
SAR 2025/026 1407
SAR 2025/029 1458
SRR 2024/008 1202
SRR 2025/002 1373
SRR 2025/016 1444
SRR 2025/015 1446
SRR 2024/039 1346
SRR 2024/040 1347
SRR 2025/008 1378
SRR 2025/009 1379
SAR 2024/016 1211
SAR 2024/061 1270
SAR 2024/049 1255
SAR 2024/0

46

# processes from names and ids

In [9]:
process_lists = [fsar_pilots, fsar_coordinators_doc_titles_approx, fsar_coordinators_process_id]

In [10]:
# pilot image tab 1
for name, abbrev in fsar_pilots:
    temp_processes = processes[
        (processes['name'].str.contains(re.escape(name))) & (processes.abbrev == abbrev)
    ]
    print(f'{abbrev:<10} {temp_processes.shape[0]:<5} {name} ')

PAC        1     Pacific Ocean Perch 
PAC        0     Salmon (7 SMUs) 
Arctic     2     Dolly Varden 
Arctic     0     EAZ/WAZ Shrimp 
QC         0     4RST Capelin 
QC         0     Unit 1 & 2 Redfish 
MAR        0     Lobster LFA 35 
GULF       0     4TVn Atlantic Cod 
GULF       0     4T Spring and Fall Herring 
GULF       0     Gulf of St. Lawrence Snow Crab 
GULF       2     Southern Gulf of St. Lawrence Scallop 
NL         9     3Ps Atlantic Cod 
NL         0     Northern Shrimp SFA 4-6 
NL         7     Northern Cod 


In [11]:
# manual database search by inspection, expanding accronyms and making best guess. 
#  -> these are not confirmed
#  typically guessed the newest one that fits the criteria
#  no meeting dates to confirm vs
possible_pilot_process_id = [
    ('Pacific Ocean Perch', 'PAC', 176),
    ('Salmon (7 SMUs)', 'PAC', 241),
    ('Dolly Varden', 'Arctic', 628),
    ('EAZ/WAZ Shrimp', 'Arctic', 629),
    ('4RST Capelin', 'QC', 356),
    ('Unit 1 & 2 Redfish', 'QC', 804),
    ('Lobster LFA 35', 'MAR', 777),
    ('4TVn Atlantic Cod', 'GULF', 648),
    ('4T Spring and Fall Herring', 'GULF', 825),
    ('Gulf of St. Lawrence Snow Crab', 'GULF', 822),
    ('Southern Gulf of St. Lawrence Scallop', 'GULF', 99),
    ('3Ps Atlantic Cod', 'NL', 840),
    ('Northern Shrimp SFA 4-6', 'NL', 645),
    ('Northern Cod', 'NL', 795),
]

In [12]:
# coordinator tab without pub number or process number 
for name, abbrev in fsar_coordinators_doc_titles_approx:
    temp_processes = processes[
        (processes['name'].str.contains(re.escape(name))) & (processes.abbrev == abbrev) 
    ]
    print(f'{abbrev:<10} {temp_processes.shape[0]:<5} {name} ')

Arctic     0     Great Slave Lake Inconnu (Stenodus leucichthys) Stock Assessment, 2022 
Arctic     0     Great Slave Lake Lake Whitefish Stock Assessment, 2022 
Arctic     0     Great Slave Lake Lake Trout Stock Assessment, 2022 
GULF       0     4T striped bass 
GULF       0     4T groundfish 
GULF       0     4T snow crab 
GULF       0     4T salmon 


In [13]:
possible_coord_doc_name_to_process_id = [
    ('Great Slave Lake Inconnu (Stenodus leucichthys) Stock Assessment, 2022', 'Arctic', 627),  # 2022 seems wrong, maybe the request, maybe that applies to 741 as well?
        # no date to confirm vs
    ('Great Slave Lake Lake Whitefish Stock Assessment, 2022', 'Arctic', 741),  # there are two that are the same (251, 741)
    ('Great Slave Lake Lake Trout Stock Assessment, 2022', 'Arctic', 741),  # no idea which is correct (or if both are wrong)
        # both the above are the same process, maybe 741?
        # 741 doesn't match meeting dates (may 22-24 2024 vs expected may 14 2024)
        # 251 even more doesn't match meeting dates (may 14-15 2023 vs expected may 14 2024)
    ('4T striped bass', 'GULF', 746),  # assume 4T means southern st lawrence, and most recent one is FSAR
        # 746 matches one of the meeting dates (dec 4 2024)
    ('4T groundfish', 'GULF', 745),
        # 745 matches one of the meeting dates (dec 11 2024)
    ('4T snow crab', 'GULF', 743),
        # 743 matches one of the meeting dates (jan 22-23 2025)
    ('4T salmon', 'GULF', 764),
        # 764 matches one of the meeting dates (feb 27 2025)
]

In [14]:
possible_process_id = list()
for x in possible_pilot_process_id:
    possible_process_id.append(x[-1])
for x in possible_coord_doc_name_to_process_id:
    possible_process_id.append(x[-1])

In [15]:
all_process_id = fsar_coordinators_process_id + possible_process_id

len(all_process_id), len(set(all_process_id))

# only a few duplicates

(32, 29)

In [16]:
doc_ids_from_fsar_processes = documents[documents.process_id.isin(all_process_id)].id.to_list()

In [17]:
[x for x in fsar_doc_id if x in doc_ids_from_fsar_processes]

[1123,
 1213,
 1229,
 1230,
 1272,
 1352,
 1376,
 1377,
 1380,
 1382,
 1398,
 1417,
 1426,
 1445,
 1458]

In [18]:
[x for x in doc_ids_from_fsar_processes if x not in fsar_doc_id]

[1422,
 1432,
 1441,
 1442,
 1443,
 1451,
 1541,
 81,
 73,
 1080,
 1414,
 1429,
 1081,
 1292,
 1293,
 1294,
 1351,
 1413,
 1423,
 1430,
 1433,
 1452,
 1453,
 1454,
 1525,
 74,
 75,
 114,
 1273,
 1124,
 1388,
 1427,
 1082,
 72,
 94,
 1397]

In [19]:
[x for x in fsar_doc_id if x not in doc_ids_from_fsar_processes]

[1172,
 1187,
 1202,
 1211,
 1250,
 1251,
 1255,
 1270,
 1274,
 1302,
 1303,
 1316,
 1317,
 1326,
 1346,
 1347,
 1348,
 1355,
 1359,
 1373,
 1378,
 1379,
 1407,
 1415,
 1416,
 1419,
 1424,
 1425,
 1444,
 1446,
 1466]

# documents linked to FSAR processes

In [20]:
docs_from_fsar_processes = documents[documents.process_id.isin(all_process_id)]

In [21]:
# some dead links, some not FSAR...

docs_from_fsar_processes.loc[
    docs_from_fsar_processes.acronym_en.isin(["SAR", "SRR"]),
    ["process_id", "id", "acronym_en", "pub_number", "url_en"]
].sort_values("process_id")

,process_id,id,acronym_en,pub_number,url_en
303,99,1272,SAR,2024/037,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_037-eng.html
278,176,1123,SAR,2024/012,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_012-eng.html
201,241,73,SAR,2022/030,http://www.dfo-mpo.ca/csas-sccs/Publications/SAR-AS/2022/2022_030-eng.html
195,356,81,SAR,2022/023,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2022/2022_023-eng.html
69,627,1541,SAR,None,None
1362,629,1229,SRR,2024/017,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ScR-RS/2024/2024_017-eng.html
1361,629,1230,SRR,2024/016,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/ScR-RS/2024/2024_016-eng.html
257,645,1080,SAR,2023/038,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2023/2023_038-eng.html
292,648,1213,SAR,2024/026,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_026-eng.html
340,743,1398,SAR,2025/010,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2025/2025_010-eng.html


# all FSAR
* from the list: fsar_doc_id

In [22]:
fsar_docs = documents[documents.id.isin(fsar_doc_id)].copy().reset_index(drop=True)

In [23]:
# check if any confirmed FSAR docs have SAR or SRR in their process that are not FSAR 
#  ie, check that the two values are the same
fsar_process_id_confirmed = set(processes[processes.id.isin(fsar_docs.process_id.to_list())].id.to_list())

fsar_docs.shape[0], documents[
    (documents.process_id.isin(fsar_process_id_confirmed)) & (documents.acronym_en.isin(["SAR", "SRR"])) 
].shape[0]
# SOOOO CLOSE!!!

(46, 47)

In [24]:
documents.loc[
    (documents.process_id.isin(fsar_process_id_confirmed)) & (documents.acronym_en.isin(["SAR", "SRR"])) 
    & (~documents.id.isin(fsar_doc_id)),
    doc_columns
]

,id,title_en,pub_number,process_id,acronym_en
320,1327,"Science Advice to Support the Rebuilding Plan for Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis)",2024/054,744,SAR


In [25]:
# check all the docs from this process
documents.loc[documents.process_id == 744, doc_columns]

,id,title_en,pub_number,process_id,acronym_en
320,1327,"Science Advice to Support the Rebuilding Plan for Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis)",2024/054,744,SAR
321,1326,"Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis) Stock Assessment to 2022",2024/055,744,SAR
974,1329,"Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis): Stock Assessment to 2022 and Rebuilding Plan Scientific Requirements",2025/008,744,RES
1011,1328,"Mitigating Bycatch of the Southern Gulf of St. Lawrence, NAFO Division 4T, of White Hake (Urophycis tenuis)",2025/045,744,RES


In [26]:
# is 1327 an FSAR? (no) is 1326? (yes)
documents.loc[documents.id.isin([1326, 1327])].T

# NO. 
#  this does not follow the fsar template:
#   https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/41266079.pdf 


,320,321
id,1327,1326
created_at,2024-08-21 17:50:44.205456,2024-08-21 17:48:21.992131
updated_at,2025-05-29 13:38:28.829646,2025-05-29 13:38:28.955988
title_en,"Science Advice to Support the Rebuilding Plan for Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis)","Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis) Stock Assessment to 2022"
title_fr,"Avis scientifique à l'appui du plan de rétablissement du stock de merluche blanche (Urophycis tenuis) dans le sud du golfe du Saint-Laurent, zone 4T de l’OPANO","Évaluation du stock de merluche blanche (Urophycis tenuis) jusqu’en 2022 dans le sud du golfe du Saint-Laurent, zone 4T de l’OPANO"
title_iku,None,None
pub_number,2024/054,2024/055
pages_en,12,9
status,12,12
old_id,NaN,NaN


In [27]:
fsar_docs[['id', 'url_en']]

,id,url_en
0,1172,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_009-eng.html
1,1123,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_012-eng.html
2,1211,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_016-eng.html
3,1187,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_020-eng.html
4,1274,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_021-eng.html
5,1213,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_026-eng.html
6,1251,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_029-eng.html
7,1250,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_030-eng.html
8,1272,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_037-eng.html
9,1255,https://www.dfo-mpo.gc.ca/csas-sccs/Publications/SAR-AS/2024/2024_049-eng.html


In [28]:
process_columns = [
    'id', 'name', 'scope', 'type', 'abbrev',
]
process_scope_choices = (
    (1, 'Regional'),
    (2, 'Zonal'),
    (3, 'National'),
    (4, 'Multi-Regional'),
)

process_type_choices = (
    (1, 'Science Advisory Meeting'),
    (2, 'Science Response Process'),
)
processes.loc[
    (processes.id == 744),
    process_columns
]

,id,name,scope,type,abbrev
41,744,"Southern Gulf of St. Lawrence, NAFO Division 4T, White Hake (Urophycis tenuis) Stock Assessment and Science Advice to Support the Rebuilding Plan",1,1,GULF


# how many of each type of doc?

In [29]:
fsar_docs['acronym_en'].value_counts()

acronym_en
SAR    27
SRR    19
Name: count, dtype: int64

In [30]:
docs_from_fsar_processes['acronym_en'].value_counts()

acronym_en
SAR    20
RES    20
SRR     8
PRO     3
Name: count, dtype: int64

# keep going? or give up?

In [31]:
# NOTES: too many inconsistencies to justify the method we implemented in the app. 
#  with this data quality, it isn't really possible to line up documents without an explicit list, or checking them all manually

# TODO: OPTIONS:

#  1A) get a complete and unique list
#    web crawler that checks all documents and examines the format for standard FSAR format
#    do it manually
#    maybe someone has a complete list?
#  and 1B) create a new filetype
#    need approval - there was pushback initially

#  2) give up, don't implement FSAR in the app, and don't track FSAR as per initial plans 

